In [4]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 3.5MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=dfa4e53f61602643f4c6436528896100eb8f1df51e97440f917d3a39fd43f6dd
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=273fe527ccb621aaa25cade3be31630520dbb0ac864d1ec7e9e16c86f5ad8efe
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [5]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [7]:
fashion_mnist=keras.datasets.fashion_mnist

In [8]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [9]:
train_images=train_images/255.0
test_images=test_images/255.0

In [10]:
train_images[0].shape

(28, 28)

In [11]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [53]:
def build_model(hp):
  model = keras.Sequential([
      keras.layers.Conv2D(
          filters=hp. Int('conv_1_filter', min_value=32, max_value=128, step=16),
          kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
          activation='relu',
          input_shape=(28,28,1)
      ),
      keras.layers.Conv2D(
          filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
          kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
          activation='relu'
      ),
      keras.layers.Flatten(),
      keras.layers.Dense(
          units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
          activation='relu'
      ),
      keras.layers.Dense(10, activation='softmax')#output layer
    ])
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model


In [64]:
from kerastuner import RandomSearch 
from kerastuner.engine.hyperparameters import HyperParameters

In [65]:
tuner_search=RandomSearch(build_model,
                           objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

INFO:tensorflow:Reloading Oracle from existing project output/Mnist Fashion/oracle.json
INFO:tensorflow:Reloading Tuner from output/Mnist Fashion/tuner0.json


In [66]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 4 Complete [00h 19m 48s]
val_accuracy: 0.9098333120346069

Best val_accuracy So Far: 0.9141666889190674
Total elapsed time: 01h 27m 28s
INFO:tensorflow:Oracle triggered exit


In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]

In [70]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 128)       1280      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 64)        204864    
_________________________________________________________________
flatten (Flatten)            (None, 30976)             0         
_________________________________________________________________
dense (Dense)                (None, 32)                991264    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                330       
Total params: 1,197,738
Trainable params: 1,197,738
Non-trainable params: 0
_________________________________________________________________
